In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import networkx as nx
import EoN
import numpy as np

## Testing meaning of recovery_rate (gamma) parameter

Setting gamma=0.07 should mean that it takes infected people 14 days to recover.

In [5]:
gamma = 0.07
G = nx.random_regular_graph(10, 1000000)
data = EoN.fast_SIR(G, 0, gamma, initial_infecteds=list(range(1000)), return_full_data=True)

# Run test
times_of_recovery = np.array([data.node_history(i)[0][1] for i in range(1000)])
np.mean(times_of_recovery)

13.203739625097958

We see that this is the case. Hence, gamma=(1/infectious period), as expected.

## Testing meaning of transmission_rate (tau) parameter